In [4]:
import plotly.graph_objects as go

from apollo.calculations.average_true_range import AverageTrueRangeCalculator
from apollo.calculations.elliot_waves_calculator import ElliotWavesCalculator
from apollo.providers.price_data_enhancer import PriceDataEnhancer
from apollo.providers.price_data_provider import PriceDataProvider
from apollo.settings import FREQUENCY, TICKER
from apollo.utils.common import to_default_date_string

In [5]:
start_date = "1995-01-01"
end_date = "2024-09-01"

window_size = 5

price_data_provider = PriceDataProvider()
price_data_enhancer = PriceDataEnhancer()

dataframe = price_data_provider.get_price_data(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

dataframe = price_data_enhancer.enhance_price_data(
    price_dataframe=dataframe,
    additional_data_enhancers=["VIX", "SP500 Futures"],
)

dataframe["prev_close"] = dataframe["adj close"].shift(1)

atr_calculator = AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
atr_calculator.calculate_average_true_range()

ew_calculator = ElliotWavesCalculator(
    dataframe=dataframe,
    window_size=window_size,
    fast_oscillator_period=5,
    slow_oscillator_period=35,
)
ew_calculator.calculate_elliot_waves()

dataframe = dataframe.loc["2007-06-01":"2007-09-01"]

dataframe

,ticker,adj close,adj high,adj low,adj open,adj volume,close,high,low,open,...,prev_close,tr,atr,high_low_avg,fast_hla_sma,slow_hla_sma,ewo,ewt,prev_ewt,ew
date,,,,,,,,,,,,,,,,,,,,,
2007-06-01,SPY,110.408318,110.637613,109.999870,110.265007,7.722541e+07,154.080002,154.399994,153.509995,153.880005,...,109.863739,0.773874,0.901653,110.318741,109.415877,107.593353,1.822524,0.0,0.0,0.0
2007-06-04,SPY,110.422646,110.630445,109.978368,110.021360,5.589836e+07,154.100006,154.389999,153.479996,153.539993,...,110.408318,0.652077,0.851738,110.304406,109.750506,107.779557,1.970949,0.0,0.0,0.0
2007-06-05,SPY,109.985603,110.279387,109.534164,110.164745,9.094496e+07,153.490005,153.899994,152.860001,153.740005,...,110.422646,0.888482,0.859087,109.906775,109.951875,107.923690,2.028186,1.0,0.0,0.0
2007-06-06,SPY,108.803223,109.598610,108.738734,109.534122,1.175860e+08,151.839996,152.949997,151.750000,152.860001,...,109.985603,1.246869,0.936643,109.168672,109.938978,108.032711,1.906268,0.0,1.0,0.0
2007-06-07,SPY,106.839828,109.276145,106.811160,108.602572,1.665401e+08,149.100006,152.500000,149.059998,151.559998,...,108.803223,2.464985,1.242311,108.043652,109.548450,108.107336,1.441113,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-08-27,SPY,105.753036,106.746161,105.594712,106.400731,8.133830e+07,146.949997,148.330002,146.729996,147.850006,...,106.746162,1.151450,1.764194,106.170437,105.441447,106.824829,-1.383383,0.0,1.0,0.0
2007-08-28,SPY,103.428566,105.249288,103.241460,105.184522,1.581731e+08,143.720001,146.250000,143.460007,146.160004,...,105.753036,2.511576,1.913671,104.245374,105.412650,106.684290,-1.271640,1.0,0.0,4.0
2007-08-29,SPY,105.458015,105.601955,103.601319,103.896369,1.494391e+08,146.539993,146.740005,143.960007,144.369995,...,103.428566,2.173389,1.965614,104.601637,105.309746,106.562258,-1.252512,0.0,1.0,0.0


In [6]:
value_1 = "adj close"
value_2 = "ew"
value_3 = "ewo"
value_4 = "adj low"

x = dataframe.index.to_numpy()

y1 = dataframe[value_1].to_numpy()
y2 = dataframe[value_2].to_numpy()
y3 = dataframe[value_3].to_numpy()
y4 = dataframe[value_4].to_numpy()

trace1 = go.Scatter(x=x, y=y1, name=value_1, yaxis="y1")
trace2 = go.Scatter(x=x, y=y2, name=value_2, yaxis="y2")
trace3 = go.Scatter(x=x, y=y3, name=value_3, yaxis="y2")
trace4 = go.Scatter(x=x, y=y4, name=value_4, yaxis="y2")

# Plot title
title = (
    f"{dataframe.iloc[0]["ticker"]}"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(
    data=[
        trace1,
        trace2,
        # trace3,
        # trace4,
    ],
    layout=layout,
)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)